<a href="https://colab.research.google.com/github/AJamal27891/University-coursees-projects/blob/master/Matching_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download dependencies

In [1]:
!rm -r  test_trainer 

In [2]:
!pip install transformers
!pip install datasets

## load datasets 

In [3]:
dirty_walmart_amazon = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_walmart_amazon'
dirty_dblp_scholar = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_scholar'
dirty_amazon_ituns = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_amazon_itunes'
dirty_dblp_acm = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/dirty_dblp_acm'
abt_buy = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/abt_buy'
amazon_google = r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/amazon_google'
QQP= r'/content/drive/MyDrive/entity-matching-transformer-master/data_DeepMatching/QQP'

## Test dataset merger functions

In [4]:
import pandas as pd 
data_A = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['samsung s5','iphone','xperia','sonny','hstc','xomi']})
data_B = pd.DataFrame({'id':[1,2,3,4,5,6],'price':[.2,.5,.6,.7,1,.8],'describtion':['iphone','xperia','sonny','huawi','xomi','samsung s5']})
lables = pd.DataFrame({'left_id':[1,2,3,4,5,6],'right_id':[6,1,2,3,4,5],'label':[1,1,1,1,0,1]})

In [5]:
total_data = lables.merge(data_A,left_on='left_id',right_on='id')
total_data = total_data.merge(data_B,left_on='right_id',right_on='id')
total_data

,left_id,right_id,label,id_x,price_x,describtion_x,id_y,price_y,describtion_y
0,1,6,1,1,0.2,samsung s5,6,0.8,samsung s5
1,2,1,1,2,0.5,iphone,1,0.2,iphone
2,3,2,1,3,0.6,xperia,2,0.5,xperia
3,4,3,1,4,0.7,sonny,3,0.6,sonny
4,5,4,0,5,1.0,hstc,4,0.7,huawi
5,6,5,1,6,0.8,xomi,5,1.0,xomi


In [6]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

,label,price_x,describtion_x,price_y,describtion_y
0,1,0.2,samsung s5,0.8,samsung s5
1,1,0.5,iphone,0.2,iphone
2,1,0.6,xperia,0.5,xperia
3,1,0.7,sonny,0.6,sonny
4,0,1.0,hstc,0.7,huawi
5,1,0.8,xomi,1.0,xomi


In [7]:
def remove_id_columns(total_data):
  for x in total_data:
    if x.endswith('id'):
      total_data.drop(x,axis=1,inplace=True)
    if x.startswith('id'):
      total_data.drop(x,axis=1,inplace=True)
  return total_data  
remove_id_columns(total_data)

def merger(data_A,data_B,train,test,valid,
           remove_ids=True,
           merge_attributes=True,
           drop_original=True,
           tuples=True):
  train_data = train.merge(data_A,left_on='ltable_id',right_on='id')
  train_data = train_data.merge(data_B,left_on='rtable_id',right_on='id')
  test_data = test.merge(data_A,left_on='ltable_id',right_on='id')
  test_data = test_data.merge(data_B,left_on='rtable_id',right_on='id')
  valid_data = valid.merge(data_A,left_on='ltable_id',right_on='id')
  valid_data = valid_data.merge(data_B,left_on='rtable_id',right_on='id')
  if remove_ids:
    train_data = remove_id_columns(train_data)
    test_data = remove_id_columns(test_data)
    valid_data = remove_id_columns(valid_data)
  if merge_attributes:
    train_data.iloc[:,1:]  = train_data.iloc[:,1:].astype(str).apply(lambda x : ' [ATR]'+x.name.strip('_y').strip('_x') +'[VAL]'+x)
    train_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : ' '.join(x.astype(str)) ,axis=1)
    train_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"[SEP]" + ' '.join(x.astype(str)) ,axis=1)
    test_data.iloc[:,1:]  = test_data.iloc[:,1:].astype(str).apply(lambda x : ' [ATR]'+x.name.strip('_y').strip('_x') +'[VAL]'+x)
    test_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : ' '.join(x.astype(str)) ,axis=1)
    test_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"[SEP]" + ' '.join(x.astype(str)) ,axis=1)
    valid_data.iloc[:,1:]  = valid_data.iloc[:,1:].astype(str).apply(lambda x : ' [ATR]'+x.name.strip('_y').strip('_x') +'[VAL]'+x)
    valid_data['text_left'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_x')]].apply(lambda x : ' '.join(x.astype(str)) ,axis=1)
    valid_data['text_right'] =train_data[[c for c in train_data.columns[1:] if c.endswith('_y')]].apply(lambda x :"[SEP]" + ' '.join(x.astype(str)) ,axis=1) 

  if drop_original:
    train_data= train_data[['label','text_left','text_right']]
    test_data= test_data[['label','text_left','text_right']]
    valid_data= valid_data[['label','text_left','text_right']]
  if tuples:
    train_data['tuples']= train_data[['text_left','text_right']].values.tolist()
    train_data['tuples'] = train_data['tuples'].apply(lambda x: '[SEP]'.join(x))
    test_data['tuples']= test_data[['text_left','text_right']].values.tolist()
    test_data['tuples'] = test_data['tuples'].apply(lambda x: '[SEP]'.join(x))
    valid_data['tuples']= valid_data[['text_left','text_right']].values.tolist()
    valid_data['tuples'] = valid_data['tuples'].apply(lambda x: '[SEP]'.join(x))

  return train_data, test_data, valid_data

# Data Preprocessing

## Amazon Google data

In [8]:
table_A = pd.read_csv(amazon_google+r'/deep_matcher/tableA.csv')
table_B = pd.read_csv(amazon_google+r'/deep_matcher/tableB.csv')
AG_train = pd.read_csv(amazon_google+r'/deep_matcher/train.csv')
AG_test = pd.read_csv(amazon_google+r'/deep_matcher/test.csv')
AG_valid = pd.read_csv(amazon_google+r'/deep_matcher/valid.csv')
AG_train, AG_test, AG_valid = merger(table_A,table_B,AG_train,AG_test,AG_valid)
AG_train.head()

,label,text_left,text_right,tuples
0,0,[ATR]title[VAL]microsoft visio standard 2007 ...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]microsoft visio standard 2007 ...
1,0,[ATR]title[VAL]adobe premiere pro cs3 upgrade...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe premiere pro cs3 upgrade...
2,0,[ATR]title[VAL]adobe indesign cs3 upgrade fro...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe indesign cs3 upgrade fro...
3,0,[ATR]title[VAL]adobe creative suite cs3 desig...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe creative suite cs3 desig...
4,0,[ATR]title[VAL]adobe creative suite cs3 desig...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe creative suite cs3 desig...


In [9]:
AG_train

,label,text_left,text_right,tuples
0,0,[ATR]title[VAL]microsoft visio standard 2007 ...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]microsoft visio standard 2007 ...
1,0,[ATR]title[VAL]adobe premiere pro cs3 upgrade...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe premiere pro cs3 upgrade...
2,0,[ATR]title[VAL]adobe indesign cs3 upgrade fro...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe indesign cs3 upgrade fro...
3,0,[ATR]title[VAL]adobe creative suite cs3 desig...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe creative suite cs3 desig...
4,0,[ATR]title[VAL]adobe creative suite cs3 desig...,[SEP] [ATR]title[VAL]adobe cs3 design standard...,[ATR]title[VAL]adobe creative suite cs3 desig...
...,...,...,...,...
6869,0,[ATR]title[VAL]x-oom internet movies 2 [ATR]...,[SEP] [ATR]title[VAL]feral interactive the mov...,[ATR]title[VAL]x-oom internet movies 2 [ATR]...
6870,1,[ATR]title[VAL]pc hoyle jewels : swap & drop ...,[SEP] [ATR]title[VAL]encore software 10662 hoy...,[ATR]title[VAL]pc hoyle jewels : swap & drop ...
6871,0,[ATR]title[VAL]autodesk discreet combustion 4...,[SEP] [ATR]title[VAL]autodesk autodesk combust...,[ATR]title[VAL]autodesk discreet combustion 4...
6872,0,[ATR]title[VAL]virtual labs : light [ATR]man...,[SEP] [ATR]title[VAL]virtual labs : light [AT...,[ATR]title[VAL]virtual labs : light [ATR]man...


In [10]:
AG_train = AG_train.groupby('label')
AG_train = AG_train.apply(lambda x: x.sample(AG_train.size().min()).reset_index(drop=True))
AG_train = AG_train.sample(frac=1).reset_index(drop=True)
AG_test = AG_test.groupby('label')
AG_test = AG_test.apply(lambda x: x.sample(AG_test.size().min()).reset_index(drop=True))
AG_test = AG_test.sample(frac=1).reset_index(drop=True)
AG_valid = AG_valid.groupby('label')
AG_valid = AG_valid.apply(lambda x: x.sample(AG_valid.size().min()).reset_index(drop=True))
AG_valid = AG_valid.sample(frac=1).reset_index(drop=True)


In [11]:
AG_train

,label,text_left,text_right,tuples
0,0,[ATR]title[VAL]cubase se3 educational edition...,[SEP] [ATR]title[VAL]zipmagic personal edition...,[ATR]title[VAL]cubase se3 educational edition...
1,1,[ATR]title[VAL]total 3d home & landscape desi...,[SEP] [ATR]title[VAL]individual software inc t...,[ATR]title[VAL]total 3d home & landscape desi...
2,1,[ATR]title[VAL]zend studio 5.1 ( win/mac ) [...,[SEP] [ATR]title[VAL]zend studio 5.1 php integ...,[ATR]title[VAL]zend studio 5.1 ( win/mac ) [...
3,1,[ATR]title[VAL]pinnacle mobile media converte...,[SEP] [ATR]title[VAL]pinnacle mobile media con...,[ATR]title[VAL]pinnacle mobile media converte...
4,0,[ATR]title[VAL]1pk internet sec ste 2007 oem ...,[SEP] [ATR]title[VAL]mcafee internet security ...,[ATR]title[VAL]1pk internet sec ste 2007 oem ...
...,...,...,...,...
1393,0,[ATR]title[VAL]adobe creative suite cs3 produ...,[SEP] [ATR]title[VAL]adobe cs3 production prem...,[ATR]title[VAL]adobe creative suite cs3 produ...
1394,0,[ATR]title[VAL]adobe creative suite cs3 desig...,[SEP] [ATR]title[VAL]adobe cs3 master collecti...,[ATR]title[VAL]adobe creative suite cs3 desig...
1395,1,[ATR]title[VAL]emedia piano and keyboard meth...,[SEP] [ATR]title[VAL]emedia music corp emedia ...,[ATR]title[VAL]emedia piano and keyboard meth...
1396,1,[ATR]title[VAL]merriam-webster 's dictionary ...,[SEP] [ATR]title[VAL]merriam-webster 's dictio...,[ATR]title[VAL]merriam-webster 's dictionary ...


In [12]:
import os 
try:
  os.mkdir('merged_data')
  os.mkdir('merged_data/amazon_google')
except FileExistsError:
  pass

AG_train.to_csv(r'merged_data/amazon_google/train.csv')
AG_test.to_csv(r'merged_data/amazon_google/test.csv')
AG_valid.to_csv(r'merged_data/amazon_google/valid.csv')



In [13]:
from datasets import load_dataset
raw_datasets = load_dataset('csv',data_files={'train':['merged_data/amazon_google/train.csv','merged_data/amazon_google/valid.csv'],
                                              'test':'merged_data/amazon_google/test.csv',
                                              })

Using custom data configuration default-00ea3e09c0c0c700


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-00ea3e09c0c0c700/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [15]:
def tokenize_function(examples):
    # examples['text_left'].head()
    # examples['tuples'] = examples[["text_left","text_right"]].apply(lambda x: (x[0],x[1]),axis=1)
    return tokenizer(examples['tuples'], padding="max_length", truncation=True,max_length=128)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [16]:
tokenized_datasets = tokenized_datasets.remove_columns('Unnamed: 0')
tokenizer.decode(tokenized_datasets['train']['input_ids'][0])

'[CLS] [ ATR ] title [ VAL ] cubase se3 educational edition [ ATR ] manufacturer [ VAL ] steinberg [ ATR ] price [ VAL ] nan [SEP] [SEP] [ ATR ] title [ VAL ] zipmagic personal edition [ ATR ] manufacturer [ VAL ] nan [ ATR ] price [ VAL ] 8. 95 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [17]:
import torch
torch.cuda.empty_cache()

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",num_train_epochs=40,per_device_train_batch_size=16)

In [20]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tuples, text_right, text_left.
***** Running training *****
  Num examples = 1866
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4680


Step,Training Loss
500,0.516300
1000,0.355300
1500,0.289200
2000,0.230800
2500,0.181200
3000,0.140600
3500,0.105200
4000,0.076100
4500,0.053400


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=4680, training_loss=0.2103075785514636, metrics={'train_runtime': 666.431, 'train_samples_per_second': 112.0, 'train_steps_per_second': 7.022, 'total_flos': 6208814206771200.0, 'train_loss': 0.2103075785514636, 'epoch': 40.0})

In [22]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tuples, text_right, text_left.
***** Running Evaluation *****
  Num examples = 468
  Batch size = 8


{'eval_f1': 0.5123152709359605,
 'eval_loss': 3.2537617683410645,
 'eval_runtime': 1.4076,
 'eval_samples_per_second': 332.475,
 'eval_steps_per_second': 41.915}